### import libraries

In [1]:
import pandas as pd
import numpy as np
from deep_translator import GoogleTranslator
import pandas_profiling as pp

pd.set_option('display.max_rows', None)

### read data

#### item_categories.csv

In [2]:
item_cat = pd.read_csv('item_categories.csv')
item_cat['item_category_name_ch'] = item_cat['item_category_name'].apply(lambda x: GoogleTranslator(source='auto', target='zh').translate(x))
item_cat['item_category_name_en'] = item_cat['item_category_name'].apply(lambda x: GoogleTranslator(source='auto', target='en').translate(x))

item_cat

,item_category_name,item_category_id,item_category_name_ch,item_category_name_en
0,PC - Гарнитуры/Наушники,0,电脑-头戴式耳机,PC - Headsets / Headphones
1,Аксессуары - PS2,1,配件-PS2,Accessories - PS2
2,Аксессуары - PS3,2,配件-PS3,Accessories - PS3
3,Аксессуары - PS4,3,配件-PS4,Accessories - PS4
4,Аксессуары - PSP,4,配件-PSP,Accessories - PSP
5,Аксессуары - PSVita,5,配件-PSVita,Accessories - PSVita
6,Аксессуары - XBOX 360,6,配件-XBOX 360,Accessories - XBOX 360
7,Аксессуары - XBOX ONE,7,配件-XBOX ONE,Accessories - XBOX ONE
8,Билеты (Цифра),8,票数,Tickets (Number)
9,Доставка товара,9,交货,Delivery of goods


A few things to do next:
- category names can be further split by dash and brackets 
- Игры fails to translate into English, and by googling I found this mean "games" so I will mannualy translate it
- Кино is translated into "movie" and "cinema", and "movie" is more appropriate so I will replace "cinema" with "movie"
- PC Games, Android Games, and MAC Games are all games, so I will replace them with "Games" and add a subcategory differentiating devices

In [3]:
# split category name by "-"
item_cat[['category0','category1']] = item_cat.item_category_name_en.str.split(" - ",expand=True)
# some category names are not split by "-" but by "()", so find those and separate by "()"
item_cat['category1'] = item_cat.apply(lambda x: x['category0'][x['category0'].find("(")+1:x['category0'].find(")")] if '(' in x['category0'] and x['category1']==None
                                      else x['category1'], axis=1)
# since the subcategory inside "()" have been extracted and placed in category1 column, remove them from category0 column
item_cat['category0'] = item_cat['category0'].apply(lambda x: x.split('(')[0].strip() if '(' in x else x)
# correct typos
item_cat['category0'].replace({'Game Consoles':'Game consoles', 
                               'Игры':'Games', 
                               'Payment Cards':'Payment cards', 
                               'Cinema':'Movie'}, 
                              inplace=True)

item_cat

,item_category_name,item_category_id,item_category_name_ch,item_category_name_en,category0,category1
0,PC - Гарнитуры/Наушники,0,电脑-头戴式耳机,PC - Headsets / Headphones,PC,Headsets / Headphones
1,Аксессуары - PS2,1,配件-PS2,Accessories - PS2,Accessories,PS2
2,Аксессуары - PS3,2,配件-PS3,Accessories - PS3,Accessories,PS3
3,Аксессуары - PS4,3,配件-PS4,Accessories - PS4,Accessories,PS4
4,Аксессуары - PSP,4,配件-PSP,Accessories - PSP,Accessories,PSP
5,Аксессуары - PSVita,5,配件-PSVita,Accessories - PSVita,Accessories,PSVita
6,Аксессуары - XBOX 360,6,配件-XBOX 360,Accessories - XBOX 360,Accessories,XBOX 360
7,Аксессуары - XBOX ONE,7,配件-XBOX ONE,Accessories - XBOX ONE,Accessories,XBOX ONE
8,Билеты (Цифра),8,票数,Tickets (Number),Tickets,Number
9,Доставка товара,9,交货,Delivery of goods,Delivery of goods,None


#### shops.csv

In [4]:
shops = pd.read_csv('shops.csv')
shops['shop_name_ch'] = shops['shop_name'].apply(lambda x: GoogleTranslator(source='auto', target='zh').translate(x))
shops['shop_name_en'] = shops['shop_name'].apply(lambda x: GoogleTranslator(source='auto', target='en').translate(x))

shops[:20]

,shop_name,shop_id,shop_name_ch,shop_name_en
0,"!Якутск Орджоникидзе, 56 фран",0,雅库茨克·奥尔德奇尼基泽，56法郎,"! Yakutsk Ordzhonikidze, 56 francs"
1,"!Якутск ТЦ ""Центральный"" фран",1,！Yakutsk TC“中央”法兰,"! Yakutsk TC ""Central"" fran"
2,"Адыгея ТЦ ""Мега""",2,阿迪格购物中心“ Mega”,"Adygea shopping center ""Mega"""
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Balashikha TRK“十月-基诺米尔”,"Balashikha TRK ""October-Kinomir"""
4,"Волжский ТЦ ""Волга Молл""",4,沃尔日斯基购物中心“伏尔加购物中心”,"Volzhsky shopping center ""Volga Mall"""
5,"Вологда ТРЦ ""Мармелад""",5,Vologda SEC“ Marmelad”,"Vologda SEC ""Marmelad"""
6,"Воронеж (Плехановская, 13)",6,沃罗涅日（Plekhanovskaya，13）,"Voronezh (Plekhanovskaya, 13)"
7,"Воронеж ТРЦ ""Максимир""",7,沃罗涅日TRC“马克西米尔”,"Voronezh TRC ""Maksimir"""
8,"Воронеж ТРЦ Сити-Парк ""Град""",8,沃罗涅什TRC城市公园“城市”,"Voronezh TRC City-Park ""City"""
9,Выездная Торговля,9,场外交易,Offsite Trade


A few things to do next:
- Extract location
- Extract shop name

In [5]:
def get_location(shop_name):
    if 'Offsite' in shop_name or 'online' in shop_name:
        return shop_name
    else:
        shop_name = shop_name.replace('! ', '')
        x, _ = shop_name.split(' ', 1)
        return x
    
shops['location'] = shops['shop_name_en'].apply(get_location)

In [6]:
shops

,shop_name,shop_id,shop_name_ch,shop_name_en,location
0,"!Якутск Орджоникидзе, 56 фран",0,雅库茨克·奥尔德奇尼基泽，56法郎,"! Yakutsk Ordzhonikidze, 56 francs",Yakutsk
1,"!Якутск ТЦ ""Центральный"" фран",1,！Yakutsk TC“中央”法兰,"! Yakutsk TC ""Central"" fran",Yakutsk
2,"Адыгея ТЦ ""Мега""",2,阿迪格购物中心“ Mega”,"Adygea shopping center ""Mega""",Adygea
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Balashikha TRK“十月-基诺米尔”,"Balashikha TRK ""October-Kinomir""",Balashikha
4,"Волжский ТЦ ""Волга Молл""",4,沃尔日斯基购物中心“伏尔加购物中心”,"Volzhsky shopping center ""Volga Mall""",Volzhsky
5,"Вологда ТРЦ ""Мармелад""",5,Vologda SEC“ Marmelad”,"Vologda SEC ""Marmelad""",Vologda
6,"Воронеж (Плехановская, 13)",6,沃罗涅日（Plekhanovskaya，13）,"Voronezh (Plekhanovskaya, 13)",Voronezh
7,"Воронеж ТРЦ ""Максимир""",7,沃罗涅日TRC“马克西米尔”,"Voronezh TRC ""Maksimir""",Voronezh
8,"Воронеж ТРЦ Сити-Парк ""Град""",8,沃罗涅什TRC城市公园“城市”,"Voronezh TRC City-Park ""City""",Voronezh
9,Выездная Торговля,9,场外交易,Offsite Trade,Offsite Trade


In [7]:
sales_train = pd.read_csv('sales_train.csv')
sample_submission = pd.read_csv('sample_submission.csv')
test = pd.read_csv('test.csv')
items = pd.read_csv('items.csv')

In [8]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [9]:
sample_submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


#### Check if all IDs in sales_train.csv also exist in test.csv

In [10]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [18]:
train = sales_train[['shop_id','item_id']].drop_duplicates()
train['in_train'] = 1
test_ = test.merge(train, on=['shop_id','item_id'], how='left')
test_['in_train'].sum()/len(test_)

0.520093370681606

In [19]:
train_ = test.merge(train, on=['shop_id','item_id'], how='right')
train_['in_train'].sum()/len(train_)

1.0

In [20]:
test_.head()

,ID,shop_id,item_id,in_train
0,0,5,5037,1.0
1,1,5,5320,NaN
2,2,5,5233,1.0
3,3,5,5232,1.0
4,4,5,5268,NaN


In [21]:
sales_train[(sales_train['shop_id']==5)&(sales_train['item_id']==5320)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day


In [25]:
sales_train['item_id'].unique().tolist()

[22154,
 2552,
 2554,
 2555,
 2564,
 2565,
 2572,
 2573,
 2574,
 2593,
 2604,
 2607,
 2609,
 2548,
 2611,
 2546,
 2515,
 2705,
 2715,
 2716,
 2719,
 2720,
 2473,
 2480,
 2731,
 2746,
 2748,
 2522,
 2614,
 2798,
 2833,
 2835,
 2836,
 2844,
 2845,
 2847,
 2848,
 2854,
 2857,
 2920,
 2795,
 2808,
 2628,
 2629,
 2631,
 2640,
 2822,
 2704,
 2703,
 2088,
 2090,
 2092,
 2100,
 2102,
 2103,
 2106,
 2108,
 2111,
 2117,
 2121,
 2122,
 2123,
 2149,
 2151,
 2160,
 2164,
 2389,
 2179,
 2186,
 2340,
 2189,
 2281,
 2321,
 2333,
 2252,
 2253,
 2254,
 2255,
 2256,
 2257,
 2258,
 2199,
 2439,
 2441,
 2461,
 2678,
 2462,
 2680,
 2682,
 2437,
 2202,
 2209,
 2220,
 2221,
 2412,
 2222,
 2415,
 2416,
 2417,
 2225,
 2235,
 2924,
 3168,
 3172,
 3174,
 3175,
 3253,
 3268,
 3269,
 3394,
 3402,
 3417,
 3432,
 3164,
 3223,
 3112,
 3128,
 3203,
 3204,
 3209,
 3210,
 3214,
 3130,
 3141,
 3147,
 3146,
 3148,
 3158,
 3429,
 3437,
 3438,
 3467,
 3468,
 3286,
 3428,
 3433,
 3434,
 3427,
 3193,
 2973,
 2974,
 2649,
 2650

In [15]:
pp.ProfileReport(sales_train)